<a href="https://colab.research.google.com/github/richbello/Alcaldia_Usme/blob/main/limpiza_excel_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from google.colab import files

# 1. Subir TODOS los Excels 2005–2020 convertidos desde PDF
uploaded = files.upload()
archivos = list(uploaded.keys())

# 2. Función para limpiar un archivo desconfigurado
def limpiar_excel(file):
    df = pd.read_excel(file)

    # Renombrar columnas según estructura observada
    df.columns = [
        "CODIGO", "NOMBRE", "No.CDP", "No.RP", "FECHA_REGISTRO",
        "BENEFICIARIO", "TIPO_REGISTRO", "No_COMPROMISO", "FECHA_INICIAL",
        "OBJETO", "VALOR_REGISTRO", "ANULACIONES", "AJUSTES",
        "VALOR_NETO", "AUTORIZACION_GIRO", "PENDIENTES", "ANIO"
    ]

    # Detectar filas válidas (cuando hay CDP o valor)
    mask_fila_valida = df["No.CDP"].notna() | df["VALOR_REGISTRO"].notna()

    filas_limpias = []
    fila_actual = None

    for i, row in df.iterrows():
        if mask_fila_valida.loc[i]:
            if fila_actual is not None:
                filas_limpias.append(fila_actual)
            fila_actual = row.copy()
        else:
            # Concatenar Beneficiario y Objeto partidos en varias filas
            if pd.notna(row["BENEFICIARIO"]):
                fila_actual["BENEFICIARIO"] = str(fila_actual["BENEFICIARIO"]) + " " + str(row["BENEFICIARIO"])
            if pd.notna(row["OBJETO"]):
                fila_actual["OBJETO"] = str(fila_actual["OBJETO"]) + " " + str(row["OBJETO"])

    if fila_actual is not None:
        filas_limpias.append(fila_actual)

    df_clean = pd.DataFrame(filas_limpias)

    # Limpiar columnas numéricas
    cols_num = ["VALOR_REGISTRO","ANULACIONES","AJUSTES",
                "VALOR_NETO","AUTORIZACION_GIRO","PENDIENTES"]
    for c in cols_num:
        df_clean[c] = df_clean[c].astype(str).str.replace(r"[^\d.]", "", regex=True).replace("", np.nan).astype(float)

    # Guardar limpio por año
    out_name = file.replace(".xlsx", "_limpio.xlsx")
    df_clean.to_excel(out_name, index=False)
    print(f"✅ Limpio: {out_name} ({len(df_clean)} filas)")
    return df_clean

# 3. Procesar todos los archivos subidos
df_total = []
for archivo in archivos:
    try:
        df_clean = limpiar_excel(archivo)
        df_total.append(df_clean)
    except Exception as e:
        print(f"⚠️ Error con {archivo}: {e}")

# 4. Consolidar todos los años 2005–2020
if df_total:
    df_final = pd.concat(df_total, ignore_index=True)
    df_final.to_excel("registros_predis_2005_2020_limpio.xlsx", index=False)
    files.download("registros_predis_2005_2020_limpio.xlsx")

    # 5. Resumen por Beneficiario y Año
    resumen = df_final.groupby(["ANIO","BENEFICIARIO"], as_index=False)[
        ["VALOR_REGISTRO","ANULACIONES","VALOR_NETO","AUTORIZACION_GIRO","PENDIENTES"]
    ].sum()

    resumen.to_excel("resumen_beneficiario_2005_2020.xlsx", index=False)
    files.download("resumen_beneficiario_2005_2020.xlsx")


Saving registros_predis_2005.xlsx to registros_predis_2005.xlsx
Saving registros_predis_2006_2020.xlsx to registros_predis_2006_2020.xlsx
⚠️ Error con registros_predis_2005.xlsx: Length mismatch: Expected axis has 20 elements, new values have 17 elements
⚠️ Error con registros_predis_2006_2020.xlsx: Length mismatch: Expected axis has 26 elements, new values have 17 elements


In [3]:
import pandas as pd
from google.colab import files

uploaded = files.upload()
archivos = list(uploaded.keys())

for archivo in archivos:
    df = pd.read_excel(archivo)
    print(f"{archivo} → {len(df.columns)} columnas")
    print(df.head(2))  # para ver las primeras filas
    print("-"*50)


Saving registros_predis_2005.xlsx to registros_predis_2005 (1).xlsx
Saving registros_predis_2006_2020.xlsx to registros_predis_2006_2020 (1).xlsx
registros_predis_2005 (1).xlsx → 20 columnas
    CODIGO PRESUPUESTAL                                 NOMBRE No.CDP No.RP  \
0  3-3-1-12-01-01-0039-         Continuar con los programas de    251   223   
1                    00  asistencia nutricional a la población    NaN   NaN   

  FECHA REGISTRO                      BENEFICIARIO Unnamed: 6  \
0     11-08-2005    INSTITUTO  DISTRITAL  PARA  LA        NaN   
1            NaN  PROTECCION  DE  LA  NIÑEZ  Y  LA        NaN   

  TIPO DE COMPROMISO NUMERO DE COMPROM. FECHA INICIAL  \
0           CONVENIO                  2    09-08-2005   
1      INTERADMINIST                NaN           NaN   

                          OBJETO  VALOR REGISTRO ANULACIONES  \
0    COMPONENTE \nDE \nCOMEDORES  621,454,384.00         .00   
1  FORTALECIMIENTO \nDE \nMANERA             NaN         NaN   

  AJUSTES 

In [6]:
import pandas as pd
import numpy as np
from google.colab import files

# Subir archivos Excel (2005–2020)
uploaded = files.upload()
archivos = list(uploaded.keys())

def limpiar_excel(file):
    df = pd.read_excel(file)

    # --- 1. Identificar hasta qué columna quedarnos (la de "AÑO") ---
    if "AÑO" in df.columns:
        idx = df.columns.get_loc("AÑO")   # posición de la columna AÑO
        df = df.iloc[:, :idx+1]           # cortar columnas extras
    else:
        raise ValueError(f"No se encontró columna 'AÑO' en {file}")

    # --- 2. Forzar a 17 columnas si sobran/faltan ---
    if len(df.columns) != 17:
        df = df.iloc[:, :17]  # quedarnos con las 17 primeras útiles

    df.columns = [
        "CODIGO", "NOMBRE", "No.CDP", "No.RP", "FECHA_REGISTRO",
        "BENEFICIARIO", "TIPO_REGISTRO", "No_COMPROMISO", "FECHA_INICIAL",
        "OBJETO", "VALOR_REGISTRO", "ANULACIONES", "AJUSTES",
        "VALOR_NETO", "AUTORIZACION_GIRO", "PENDIENTES", "ANIO"
    ]

    # --- 3. Reconstrucción de filas partidas ---
    mask_fila_valida = df["No.CDP"].notna() | df["VALOR_REGISTRO"].notna()
    filas_limpias, fila_actual = [], None

    for i, row in df.iterrows():
        if mask_fila_valida.loc[i]:
            if fila_actual is not None:
                filas_limpias.append(fila_actual)
            fila_actual = row.copy()
        else:
            if pd.notna(row["BENEFICIARIO"]):
                fila_actual["BENEFICIARIO"] = str(fila_actual["BENEFICIARIO"]) + " " + str(row["BENEFICIARIO"])
            if pd.notna(row["OBJETO"]):
                fila_actual["OBJETO"] = str(fila_actual["OBJETO"]) + " " + str(row["OBJETO"])

    if fila_actual is not None:
        filas_limpias.append(fila_actual)

    df_clean = pd.DataFrame(filas_limpias)

    # --- 4. Limpiar numéricos ---
    cols_num = ["VALOR_REGISTRO","ANULACIONES","AJUSTES",
                "VALOR_NETO","AUTORIZACION_GIRO","PENDIENTES"]
    for c in cols_num:
        df_clean[c] = df_clean[c].astype(str).str.replace(r"[^\d.]", "", regex=True).replace("", np.nan).astype(float)

    return df_clean


# --- 5. Procesar todos los archivos ---
df_total = []
for archivo in archivos:
    try:
        df_clean = limpiar_excel(archivo)
        df_total.append(df_clean)
        print(f"✅ Limpio: {archivo} ({len(df_clean)} filas)")
    except Exception as e:
        print(f"⚠️ Error con {archivo}: {e}")

# --- 6. Consolidado general ---
if df_total:
    df_final = pd.concat(df_total, ignore_index=True)
    df_final.to_excel("registros_predis_2005_2020_limpio.xlsx", index=False)
    files.download("registros_predis_2005_2020_limpio.xlsx")

    # --- 7. Resumen por beneficiario y año ---
    resumen = df_final.groupby(["ANIO","BENEFICIARIO"], as_index=False)[
        ["VALOR_REGISTRO","ANULACIONES","VALOR_NETO","AUTORIZACION_GIRO","PENDIENTES"]
    ].sum()

    resumen.to_excel("resumen_beneficiario_2005_2020.xlsx", index=False)
    files.download("resumen_beneficiario_2005_2020.xlsx")


Saving registros_predis_2005.xlsx to registros_predis_2005 (2).xlsx
Saving registros_predis_2006_2020.xlsx to registros_predis_2006_2020 (2).xlsx
⚠️ Error con registros_predis_2005 (2).xlsx: No se encontró columna 'AÑO' en registros_predis_2005 (2).xlsx
⚠️ Error con registros_predis_2006_2020 (2).xlsx: could not convert string to float: '.'


In [7]:
import pandas as pd

df2005 = pd.read_excel("registros_predis_2005 (2).xlsx")
print(df2005.columns.tolist())


['CODIGO PRESUPUESTAL', 'NOMBRE', 'No.CDP', 'No.RP', 'FECHA REGISTRO', 'BENEFICIARIO', 'Unnamed: 6', 'TIPO DE COMPROMISO', 'NUMERO DE COMPROM.', 'FECHA INICIAL', 'OBJETO', 'VALOR REGISTRO', 'ANULACIONES', 'AJUSTES O REINTEGROS', 'VALOR NETO', 'AUTORIZACION DE GIRO', 'COMPROMISOS SIN AUTORIZACION DE GIRO', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19']


In [10]:
import pandas as pd
import numpy as np
from google.colab import files

uploaded = files.upload()
archivos = list(uploaded.keys())

def limpiar_excel(file):
    df = pd.read_excel(file)

    # Normalizar encabezados
    df.columns = df.columns.str.strip().str.upper()

    # --- Detectar columna AÑO ---
    col_anio = [c for c in df.columns if "AÑO" in c]

    if col_anio:
        idx = df.columns.get_loc(col_anio[0])
        df = df.iloc[:, :idx+1]   # cortar hasta AÑO
    else:
        # Si no hay AÑO, cortar hasta COMPROMISOS...
        if "COMPROMISOS SIN AUTORIZACION DE GIRO" in df.columns:
            idx = df.columns.get_loc("COMPROMISOS SIN AUTORIZACION DE GIRO")
            df = df.iloc[:, :idx+1]

        # Crear columna ANIO con el año detectado del nombre del archivo
        anio = "".join([c for c in file if c.isdigit()])
        df["ANIO"] = int(anio) if anio else None

    # --- Forzar a 17 columnas ---
    if len(df.columns) > 17:
        df = df.iloc[:, :17]
    elif len(df.columns) < 17:
        while len(df.columns) < 17:
            df[f"EXTRA_{len(df.columns)}"] = None

    df.columns = [
        "CODIGO", "NOMBRE", "No.CDP", "No.RP", "FECHA_REGISTRO",
        "BENEFICIARIO", "TIPO_REGISTRO", "No_COMPROMISO", "FECHA_INICIAL",
        "OBJETO", "VALOR_REGISTRO", "ANULACIONES", "AJUSTES",
        "VALOR_NETO", "AUTORIZACION_GIRO", "PENDIENTES", "ANIO"
    ]

    # --- Reconstrucción de filas partidas ---
    mask_fila_valida = df["No.CDP"].notna() | df["VALOR_REGISTRO"].notna()
    filas_limpias, fila_actual = [], None

    for i, row in df.iterrows():
        if mask_fila_valida.loc[i]:
            if fila_actual is not None:
                filas_limpias.append(fila_actual)
            fila_actual = row.copy()
        else:
            if pd.notna(row["BENEFICIARIO"]):
                fila_actual["BENEFICIARIO"] = str(fila_actual["BENEFICIARIO"]) + " " + str(row["BENEFICIARIO"])
            if pd.notna(row["OBJETO"]):
                fila_actual["OBJETO"] = str(fila_actual["OBJETO"]) + " " + str(row["OBJETO"])

    if fila_actual is not None:
        filas_limpias.append(fila_actual)

    df_clean = pd.DataFrame(filas_limpias)

    # --- Limpieza de numéricos ---
    cols_num = ["VALOR_REGISTRO","ANULACIONES","AJUSTES",
                "VALOR_NETO","AUTORIZACION_GIRO","PENDIENTES"]

    for c in cols_num:
        df_clean[c] = (
            df_clean[c]
            .astype(str)
            .str.strip()
            .replace({".": np.nan, "-": np.nan, "": np.nan, "nan": np.nan}, regex=False)
            .str.replace(r"[^\d.,]", "", regex=True)   # eliminar texto raro
            .str.replace(",", "", regex=False)         # quitar comas de miles
        )
        df_clean[c] = pd.to_numeric(df_clean[c], errors="coerce")

    return df_clean


# Procesar todos los archivos
df_total = []
for archivo in archivos:
    try:
        df_clean = limpiar_excel(archivo)
        df_total.append(df_clean)
        print(f"✅ Limpio: {archivo} ({len(df_clean)} filas)")
    except Exception as e:
        print(f"⚠️ Error con {archivo}: {e}")

# Consolidar y guardar
if df_total:
    df_final = pd.concat(df_total, ignore_index=True)
    df_final.to_excel("registros_predis_2005_2020_limpio.xlsx", index=False)
    files.download("registros_predis_2005_2020_limpio.xlsx")

    resumen = df_final.groupby(["ANIO","BENEFICIARIO"], as_index=False)[
        ["VALOR_REGISTRO","ANULACIONES","VALOR_NETO","AUTORIZACION_GIRO","PENDIENTES"]
    ].sum()

    resumen.to_excel("resumen_beneficiario_2005_2020.xlsx", index=False)
    files.download("resumen_beneficiario_2005_2020.xlsx")


Saving registros_predis_2005.xlsx to registros_predis_2005 (5).xlsx
Saving registros_predis_2006_2020.xlsx to registros_predis_2006_2020 (5).xlsx
✅ Limpio: registros_predis_2005 (5).xlsx (1353 filas)
✅ Limpio: registros_predis_2006_2020 (5).xlsx (37779 filas)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>